In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [138]:
from sklearn.datasets import load_boston, load_diabetes
from sklearn.model_selection import train_test_split

# house_prices = load_boston()
# X, y = house_prices.data, house_prices.target

diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

n_train = 10
n_test = X.shape[0] - n_train

# normalize X
#X = (X - np.mean(X, axis=0, keepdims=True)) / np.std(X, axis=0, keepdims=True) 

# low rank approximation of X
# rank = np.linalg.matrix_rank(X)
# u, s, vh = np.linalg.svd(X)
# X = u[:,:rank].dot(np.diag(s)[:rank,:rank]).dot(vh[:rank,:])

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, test_size=n_test, random_state=4803)

# train theta
lambda_ = 0
theta = np.linalg.inv(X_train.T.dot(X_train)+lambda_*np.eye(X_train.shape[1])).dot(X_train.T.dot(y_train))

# test
y_pred = X_test.dot(theta)

# evaluate performance
print('MSE on Training Data: {:0.3f}'.format(np.sum((X_train.dot(theta) - y_train)**2)/y_train.size))
print('MSE on Test Data: {:0.3f}'.format(np.sum((y_pred - y_test)**2)/y_test.size))



MSE on Training Data: 0.000
MSE on Test Data: 99036.313


In [156]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

class MyLeastSquares:
    def __init__(self, X_train, y_train):
        """Function stores feature matrix and corresponding target data.
        
        Parameters:
        -----------
        X_train: array_like, shape(N,D)
            ndarray containing N training examples, each with D feature values.
            
        y_train: array_like, shape(N,1)
            ndarray containing target values for each of N examples in X_train."""
        
        self.X_train = X_train
        self.y_train = y_train
        
    def fit(self):
        """Function computes the weight vector of shape (D+1, 1) for regression"""
        # append ones for bias
        X = np.concatenate((self.X_train, np.ones((self.X_train.shape[0],1))), axis=1)
        self.theta = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(self.y_train))
     
    def predict(self, X_test):
        """Function predicts targets for given X_test.
        
        Parameters:
        -----------
        X_test: array_like, shape(N,D)
            ndarray containing N test examples with D features each.
            
        Returns: array_like, shape(N,1).
            ndarray containing predicted targets of shape (N,1).
        """
        
        X = np.concatenate((X_test, np.ones((X_test.shape[0],1))), axis=1)
        y_pred = X.dot(self.theta)
        
        return y_pred
    
# load dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# train-test split
n_train = 40
n_test = X.shape[0] - n_train

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, test_size=n_test, random_state=4803)

# train theta
LS = MyLeastSquares(X_train, y_train)
LS.fit()

# test
y_pred = LS.predict(X_test)

# evaluate performance
print('MSE on Training Data: {:0.3f}'.format(np.sum((LS.predict(X_train) - y_train)**2)/y_train.size))
print('MSE on Test Data: {:0.3f}'.format(np.sum((y_pred - y_test)**2)/y_pred.size))



MSE on Training Data: 3203.340
MSE on Test Data: 3282.631


In [200]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

def rank_truncate(X, rank):
    """Function perfroms rank truncation for given feature matrix X.
    
    Parameters:
    -----------
    X: array_like, shape(N,D).
        ndarray containing N training examples with D features each.
    
    rank: int
        integer specifying the number of singular values to truncate with.
        
    Returns:
    --------
    X_trunc: array_like, shape(N,D)
        ndarray containing rank-approximation of X.
    """
    
    u, s, vh = np.linalg.svd(X)
    X_trunc = u[:,:rank].dot(np.diag(s)[:rank,:rank]).dot(vh[:rank,:])
    
    return X_trunc
    
    
# load dataset
house_prices = load_boston()
X, y = house_prices.data, house_prices.target

# train-test split
n_train = 10
n_test = X.shape[0] - n_train

# perform rank truncation
rank = np.linalg.matrix_rank(X)
X = rank_truncate(X, rank)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, test_size=n_test, random_state=4803)

# train theta
LS = MyLeastSquares(X_train, y_train)
LS.fit()

# test
y_pred = LS.predict(X_test)

# evaluate performance
print('MSE on Training Data: {:0.3f}'.format(np.sum((LS.predict(X_train) - y_train)**2)/y_train.size))
print('MSE on Test Data: {:0.3f}'.format(np.sum((y_pred - y_test)**2)/y_pred.size))



MSE on Training Data: 2466.863
MSE on Test Data: 67647433114498321506257296424960.000


In [234]:
from sklearn.datasets import load_linnerud
from sklearn.model_selection import train_test_split

def gen_polynomial_features(X, power):
    """Function generates polynomial features of specified power and appends to X.
    
    Parameters:
    -----------
    X: array_like, shape(N,D).
        ndarray of feature matrix containing N training examples with D features each.
        
    power: int
        integer specifying power X is raised to before appended to itself.
    
    Returns:
    --------
    X_poly: array_like, shape(N,2D)
        ndarray of appended polynomial features to original feature matrix"""
    
    X_poly = np.concatenate((X, X**power), axis=1)
    
    return X_poly
    
# load dataset
exercises = load_linnerud()
X, y = exercises.data, exercises.target[:,0]

# train-test split
n_train = 3
n_test = X.shape[0] - n_train

#generate polynomial features
#X = gen_polynomial_features(X, power=2)

# perform rank truncation
rank = np.linalg.matrix_rank(X)
X = rank_truncate(X, rank)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, test_size=n_test, random_state=4803)

# train theta
LS = MyLeastSquares(X_train, y_train)
LS.fit()

# test
y_pred = LS.predict(X_test)

# evaluate performance
print('MSE on Training Data: {:0.3f}'.format(np.sum((LS.predict(X_train) - y_train)**2)/y_train.size))
print('MSE on Test Data: {:0.3f}'.format(np.sum((y_pred - y_test)**2)/y_pred.size))



MSE on Training Data: 870.333
MSE on Test Data: 2417.588


In [380]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

class MyRidgeLeastSquares:
    def __init__(self, X_train, y_train, alpha=0):
        """Function stores feature matrix and corresponding target data.
        
        Parameters:
        -----------
        X_train: array_like, shape(N,D)
            ndarray containing N training examples, each with D feature values.
            
        y_train: array_like, shape(N,1)
            ndarray containing target values for each of N examples in X_train.
        
        alpha: float
            float specifying the multiplier of the L2 penalty in Ridge loss"""
        
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha
        
    def fit(self):
        """Function computes the weight vector of shape (D+1, 1) for regression"""
        # append ones for bias
        X = np.concatenate((self.X_train, np.ones((self.X_train.shape[0],1))), axis=1)
        self.theta = np.linalg.inv(X.T.dot(X) + self.alpha*np.eye(X.shape[1])).dot(X.T.dot(self.y_train))
     
    def predict(self, X_test):
        """Function predicts targets for given X_test.
        
        Parameters:
        -----------
        X_test: array_like, shape(N,D)
            ndarray containing N test examples with D features each.
            
        Returns: array_like, shape(N,1).
            ndarray containing predicted targets of shape (N,1).
        """
        
        X = np.concatenate((X_test, np.ones((X_test.shape[0],1))), axis=1)
        y_pred = X.dot(self.theta)
        
        return y_pred
    
# load dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# train-test split
n_train = 10
n_test = X.shape[0] - n_train

# perform rank truncation
rank = np.linalg.matrix_rank(X)
X = rank_truncate(X, rank)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, test_size=n_test, random_state=4803)

# train theta
LS = MyRidgeLeastSquares(X_train, y_train, alpha=2)
LS.fit()

# test
y_pred = LS.predict(X_test)

# evaluate performance
print('MSE on Training Data: {:0.3f}'.format(np.sum((LS.predict(X_train) - y_train)**2)/y_train.size))
print('MSE on Test Data: {:0.3f}'.format(np.sum((y_pred - y_test)**2)/y_pred.size))



MSE on Training Data: 9599.535
MSE on Test Data: 5594.086


In [384]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

class MyLassoLeastSquares:
    def __init__(self, X_train, y_train, beta=0):
        """Function stores feature matrix and corresponding target data.
        
        Parameters:
        -----------
        X_train: array_like, shape(N,D)
            ndarray containing N training examples, each with D feature values.
            
        y_train: array_like, shape(N,1)
            ndarray containing target values for each of N examples in X_train.
        
        beta: float
            float specifying the multiplier of the L1 penalty in Ridge loss"""
        
        self.X_train = X_train
        self.y_train = y_train
        self.beta = beta
        self.theta = np.random.randn(X_train.shape[1]+1, 1)
        
    def gradient(self, X):
        """Function computes gradient of Lasso cost.
        
        Parameters:
        -----------
        X: array_like, shape(N, D+1)
            feature matrix containing N training examples containing D+1 features each
        
        Returns:
        --------
        grad = array_like, shape(D+1, 1).
            gradient vector containing gradient for each element in theta vector.
        """
        
        lasso_grad = np.ones(self.theta.shape)
        lasso_grad[self.theta<=0] = -1
        grad = (2*X.T.dot(X.dot(self.theta) - self.y_train.reshape(-1,1)))/X.shape[0] + self.beta*lasso_grad
        
        return grad
        
    
    def fit(self, num_epochs, step):
        """Function computes the weight vector of shape (D+1, 1) for regression.
        
        Parameters:
        -----------
        num_epochs: int,
            integer specifying number of trianing epochs
            
        step: float
            float specifying step size for gradient descent
        """
                
        # append ones for bias
        X = np.concatenate((self.X_train, np.ones((self.X_train.shape[0],1))), axis=1)
        
        for i in range(num_epochs):
            cost = np.sum((y_train-X.dot(self.theta))**2)/X.shape[0] + self.beta*np.sum(np.abs(self.theta))
#             self.theta = self.theta - step*self.gradient(X) # simple gradient descent
            
            
#             for j in range(self.theta.size):
#                 self.theta[j] = self.theta[j] - step*self.gradient(X)[j]                                  # coordinate descent
                
            self.theta = self.theta - step*np.linalg.inv(2/X.shape[0] * X.T.dot(X)).dot(self.gradient(X))  # newton's method
            
            if i%50==0:
                print('Cost: {:0.3f} | theta norm: {:0.3f}'.format(cost, np.linalg.norm(self.theta)))

                
                
    def predict(self, X_test):
        """Function predicts targets for given X_test.
        
        Parameters:
        -----------
        X_test: array_like, shape(N,D)
            ndarray containing N test examples with D features each.
            
        Returns: array_like, shape(N,1).
            ndarray containing predicted targets of shape (N,1).
        """
        
        X = np.concatenate((X_test, np.ones((X_test.shape[0],1))), axis=1)
        y_pred = X.dot(self.theta)
        
        return y_pred
    
# load dataset
diabetes = load_boston()
X, y = diabetes.data, diabetes.target

# train-test split
n_train = 20
n_test = X.shape[0] - n_train

# perform rank truncation
rank = np.linalg.matrix_rank(X)
X = rank_truncate(X, rank)

# normalize X
X = (X - np.mean(X, axis=0, keepdims=True)) / np.std(X, axis=0, keepdims=True) 
y = (y - y.mean()) / y.std()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, test_size=n_test, random_state=4803)

# train theta
LS = MyLassoLeastSquares(X_train, y_train, beta=0.1)
LS.fit(num_epochs=2000, step=0.5)

# LS = MyLeastSquares(X_train, y_train)
# LS.fit()

# test
y_pred = LS.predict(X_test)

# evaluate performance
print('MSE on Training Data: {:0.3f}'.format(np.sum((LS.predict(X_train) - y_train)**2)/y_train.size))
print('MSE on Test Data: {:0.3f}'.format(np.sum((y_pred - y_test)**2)/y_pred.size))



Cost: 351.918 | theta norm: 2.896
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.865 | theta norm: 5.171
Cost: 56.764 | theta norm: 5.187
Cost: 56.

In [309]:
temp = np.random.randn(3,1)
temp2 = np.ones(temp.shape)
print(temp, '\n',temp2)
temp2[temp<=0] = -1
print(temp2)

[[-1.73011186]
 [ 1.5612985 ]
 [-0.48763894]] 
 [[1.]
 [1.]
 [1.]]
[[-1.]
 [ 1.]
 [-1.]]
